In [ ]:
import subprocess
import os
import shutil
import numpy as np
import pandas as pd

In [ ]:
none_bind_df=pd.read_csv("none.csv")
none_names = none_bind_df.none_genmer_names

In [ ]:
path="/home/cptd/Documents/Molclus/fangbo"
molculs_path = f"/home/cptd/Documents/Molclus/molclus_1.12_Linux"
new_file_name = 'traj.xyz'
destination_folder = 'genmer_mol'
destination_file_path=os.path.join(path, destination_folder)

In [ ]:
def get_file_names_in_directory(path):
    files = []
    file_names =[]
    files_names = []
    for file in os.listdir(path):
        files_names.append(file)
        file_name = file.split(".")[0]
        file_names.append(file_name)
        if os.path.isfile(os.path.join(path, file)):
            files.append(os.path.join(path, file))
    return files,file_names,files_names

In [ ]:
def get_atom_num(genmer_files_names,i):
    mol1 = genmer_files_names[i].split("_")[0]
    mol2 = genmer_files_names[i].split("_")[1].split(".")[0]
    atom1 = df.loc[df['names'] == mol1, 'atom_num'].values[0]
    atom2 = df.loc[df['names'] == mol2, 'atom_num'].values[0]
    line1 = f"1-{atom1}"
    line2 = f"{atom1+1}-{atom1+atom2}"
    atomnum=atom1+atom2
    return line1,line2,atomnum

In [ ]:
def Ti_file_xyz(input_file,output_file,atomnum): 
    with open(input_file, 'r') as file:
        lines = file.readlines()
    header_lines = lines[:atomnum+2]
    with open(output_file, 'w') as new_file:
        new_file.writelines(header_lines)   

In [ ]:
def shell_change(line1,line2,i):
    with open('multiwfn_bind.sh', 'r') as file:
        lines = file.readlines()
    lines[0] = f'Multiwfn {genmer_names[i]}_best1.xyz > /dev/null << EOF\n'
    lines[8] = f'{line1}\n'
    lines[9] = f'{line2}\n'
    with open('modified_multiwfn.sh', 'w') as new_file:
        new_file.writelines(lines)
    print("The new Shell script file is successfully modified and saved")    

In [ ]:
genmer_file_paths,genmer_names,genmer_files_names = get_file_names_in_directory(destination_file_path)

In [ ]:
df = pd.read_excel("atom_num_name.xlsx")

In [ ]:
from tqdm import tqdm

In [ ]:
none_names_ok=list(none_names)

In [ ]:
genmer_file_paths_ok=[]
for i in genmer_file_paths:
    genmer_name = i.split("/")[-1].split(".")[0]
    if genmer_name in none_names_ok:
        genmer_file_paths_ok.append(i)

In [ ]:
genmer_files_names_ok = []
for i in genmer_files_names:
    genmer_name = i.split(".")[0]
    if genmer_name in none_names_ok:
        genmer_files_names_ok.append(i)

In [ ]:
genmer_file_paths=genmer_file_paths_ok
genmer_names=none_names_ok
genmer_files_names=genmer_files_names_ok

In [ ]:
#Create a new file path
new_folder_path = f'{molculs_path}/408'
os.makedirs(new_folder_path, exist_ok=True)

In [ ]:
for i in tqdm(range(len(genmer_files_names))):
    line1,line2,atomnum=get_atom_num(genmer_files_names,i)
    destination_file = os.path.join(molculs_path, new_file_name)
    shutil.copy(genmer_file_paths[i], destination_file)
    os.rename(destination_file, os.path.join(molculs_path, new_file_name))     
    os.chdir(molculs_path)
    os.system("bash molclus.sh")
    # 读取原始XYZ文件
    input_file='cluster.xyz'
    output_file=f'{genmer_names[i]}_best1.xyz'
    Ti_file_xyz(input_file,output_file,atomnum)
    shell_change(line1,line2,i)
    os.system("bash modified_multiwfn.sh")
    sub_folder_path = os.path.join(new_folder_path, f'{genmer_names[i]}')
    try:
        os.makedirs(sub_folder_path,exist_ok=True)
    except:
        pass
    #SP_path = os.path.join(sub_folder_path,f"{genmer_names[i]}")
    shutil.move("SP.inp", sub_folder_path)

In [ ]:
# Specify the path of the directory to traverse
root_folder = new_folder_path
SP_paths=[]
Out_paths=[]
gbw_paths=[]
# Iterate through all folders and files in a given directory
for root, dirs, files in os.walk(root_folder):
    for file in files:
        file_out_path=os.path.join(root, "SP.out")
        gbw_paths.append(os.path.join(root, "SP.gbw"))
        file_path = os.path.join(root, file)
        SP_paths.append(file_path)
        Out_paths.append(file_out_path)

In [ ]:
error_SP_path=[]
error_OUT_path=[]
num=1
for i in tqdm(range(len(SP_paths))):
    SP_file = SP_paths[i]
    Out_file = Out_paths[i]
    num+=1
    os.system(f"/home/cptd/Software/ORCA-5.0.3/orca {SP_file} > {Out_file}")
print(num)
        #os.system(f"/home/cptd/Software/ORCA-5.0.3/orca {SP_file} > {Out_file}")
    #try:
       # os.system(f"orca {SP_file} > {Out_file}")
    #except:
        #error_path.append(SP_file)